In [136]:
%matplotlib inline

import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm
# from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import warnings
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)
warnings.filterwarnings('ignore')

In [137]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import torchvision.models as models
from torchsummary import summary

In [138]:
train = pd.read_hdf('/home/sunnyu/yuchuan/data/train.h5')
test = pd.read_hdf('/home/sunnyu/yuchuan/data/test.h5')

In [139]:
def map_feature(train):
    m,n=4,4
    df=train
    ymax,ymin,xmax,xmin=df['y'].max(),df['y'].min(),df['x'].max(),df['x'].min()
    yy=(df['y']-ymin)/(ymax-ymin)
    xx=(df['x']-xmin)/(xmax-xmin)
    count,count10,vmean,dmean=[],[],[],[]
    for i in range(m):
        count.append([]),count10.append([]),vmean.append([]),dmean.append([])
        for j in range(n):
            if ((i==m)|(j==n)):
                dfij=df[((i/m)<=yy) & (yy<=((i+1)/m)) & ((j/n)<=xx) & (xx<=((j+1)/n))]
            else:
                dfij=df[((i/m)<=yy) & (yy<((i+1)/m)) & ((j/n)<=xx) & (xx<((j+1)/n))]
            cc=dfij.shape[0]
            count[i].append(cc)
            count10[i].append(cc!=0)
            vmean[i].append(dfij['v'].mean())
            dmean[i].append(dfij['d'].mean())
    map_f=np.array([np.array(count),np.array(count10),np.array(vmean),np.array(dmean)]).reshape(m*n*4).T
    return pd.DataFrame(map_f)

In [ ]:
train_p=train.groupby(['ship']).apply(map_feature).unstack()
test_p=test.groupby(['ship']).apply(map_feature).unstack()

In [ ]:
train_p.to_hdf('/home/sunnyu/yuchuan/data/train_cnn_4.h5', 'df', mode='w')
test_p.to_hdf('/home/sunnyu/yuchuan/data/test_cnn_4.h5', 'df', mode='w')
del train_p,test_p

In [172]:
train_cnn = pd.read_hdf('/home/sunnyu/yuchuan/data/train_cnn_4.h5')
test_cnn = pd.read_hdf('/home/sunnyu/yuchuan/data/test_cnn_4.h5')

In [173]:
train_cnn.fillna(0,inplace=True)
test_cnn.fillna(0,inplace=True)

In [52]:
from sklearn.model_selection import train_test_split 

In [164]:
y_train=train.groupby(['ship'])['type'].first()
type_map = dict(zip(y_train.unique(), np.arange(3)))
y_train=y_train.map(type_map)
type_map_rev = {v:k for k,v in type_map.items()}

x_train, x_val, y_train, y_val = train_test_split(train_cnn,y_train, 
                                                    test_size=0.2, random_state=42)

y_train=torch.tensor(y_train.values)
x_train=torch.Tensor(x_train.values).reshape(5600,4,4,4)

y_val=torch.tensor(y_val.values)
x_val=torch.Tensor(x_val.values).reshape(1400,4,4,4)


# x_train=torch.cat((x_train,
#                    x_train[y_train==1],x_train[y_train==1][:914],
#                    x_train[y_train==2],x_train[y_train==2],
#                    x_train[y_train==2],x_train[y_train==2][:216]),axis=0)
# y_train=torch.cat((y_train,
#                    y_train[y_train==1],y_train[y_train==1][:914],
#                    y_train[y_train==2],y_train[y_train==2],
#                    y_train[y_train==2],y_train[y_train==2][:216]),axis=0)

train_data=torch.utils.data.TensorDataset(x_train,y_train)
validation_data=torch.utils.data.TensorDataset(x_val,y_val)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=128)
validation_loader = torch.utils.data.DataLoader(validation_data , batch_size=1400)

x_test=torch.Tensor(test_cnn.values).reshape(2000,4,4,4)

In [149]:
summary(model,(4,4,4))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1              [-1, 4, 4, 4]               8
            Conv2d-2             [-1, 16, 1, 1]           1,040
              ReLU-3             [-1, 16, 1, 1]               0
            Linear-4                    [-1, 3]              51
Total params: 1,099
Trainable params: 1,099
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


In [511]:
y_train=train.groupby(['ship'])['type'].first()
type_map = dict(zip(y_train.unique(), np.arange(3)))
y_train=y_train.map(type_map)
type_map_rev = {v:k for k,v in type_map.items()}


y_train=torch.tensor(y_train.values)
x_train=torch.Tensor(train_cnn.values).reshape(7000,4,4,4)

train_data=torch.utils.data.TensorDataset(x_train,y_train)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=128)

x_test=torch.Tensor(test_cnn.values).reshape(2000,4,4,4)

In [176]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.BatchNorm2d(4),
            nn.Conv2d(4, 16, 4, 1,),
            nn.ReLU(),
#             nn.MaxPool2d(2, 2) 
        )
#         self.conv2 = nn.Sequential(
#             nn.Conv2d(8, 16, 3),
#             nn.ReLU(),
#         )
        self.fc1 = nn.Linear(16, 3)
 
    # 定义前向传播过程，输入为x
    def forward(self, x):
        per_out=[]
        x = self.conv1(x)
#         x = self.conv2(x)
        x = x.view(x.size()[0], -1)
        per_out.append(x)
        x = self.fc1(x)

        return x,per_out

In [177]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
model =  Net().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

In [184]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [185]:
num_epochs = 300
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_index, (inputs,labels) in enumerate(train_loader):
        optimizer.zero_grad()
        inputs=inputs.to(device)
        labels=labels.to(device)
        outputs = model(inputs)[0]
        loss = criterion(outputs,labels)  # 计算loss
        loss.backward()     # loss 求导
        optimizer.step()
        
        running_loss += loss.item()  
        
    print(running_loss/7000)  

print('Finished Training')

0.004378601849079132
0.004371167957782745
0.004370763229472297
0.004370439308030265
0.004370149944509779
0.004369944836412157
0.004369686267205647
0.004369599802153451
0.004369369434458869
0.004369258591106959
0.0043691123723983765
0.004369019333805357
0.0043688625693321225
0.004368752517870494
0.004368657056774412
0.004368539090667452
0.004368401442255292
0.004368361788136619
0.004368214751992907
0.004368164509534836
0.004368018350430897
0.004367974098239626
0.004367852934769221
0.004367781132459641
0.004367672430617468
0.004367612787655422
0.004367513294730867
0.0043674573429993225
0.004367373492036547
0.004367219367197582
0.004367187474455152
0.004367042950221471
0.00436713576742581
0.004366919862372535
0.004366918453148433
0.00436675175172942
0.004366788868393217
0.004366687438317708
0.0043665571340492796
0.004366531423160008
0.00436651108946119
0.004366346738168172
0.004366317246641432
0.004366282497133528
0.004366165663514819
0.004366077959537506
0.004366077555077417
0.0043660150

In [186]:
yy_pred,yy_ture=torch.Tensor([[0,0,0]]),torch.tensor([0])
for batch_index, (inputs,labels) in enumerate(train_loader):
    inputs=inputs.to(device)
    labels=labels.to(device)
    outputs = model(inputs)[0].cpu()
    labels=labels.cpu()
    yy_ture=torch.cat((yy_ture,labels),axis=0)
    yy_pred=torch.cat((yy_pred,outputs),axis=0)
yy_pred=yy_pred.detach().numpy()[1:]
yy_ture=yy_ture.detach().numpy()[1:]
y_pred=np.argmax(yy_pred,axis=1)
metrics.f1_score(yy_ture,y_pred,average='macro')

0.6665484797099547

In [169]:
yy_pred,yy_ture=torch.Tensor([[0,0,0]]),torch.tensor([0])
for batch_index, (inputs,labels) in enumerate(validation_loader):
    inputs=inputs.to(device)
    labels=labels.to(device)
    outputs = model(inputs)[0].cpu()
    labels=labels.cpu()
    yy_ture=torch.cat((yy_ture,labels),axis=0)
    yy_pred=torch.cat((yy_pred,outputs),axis=0)
yy_pred=yy_pred.detach().numpy()[1:]
yy_ture=yy_ture.detach().numpy()[1:]
y_pred=np.argmax(yy_pred,axis=1)
metrics.f1_score(yy_ture,y_pred,average='macro')

0.3775268657701909

In [132]:
outputs=model(x_test.cuda())
pred=outputs[0].cpu().detach().numpy()
pred = np.argmax(pred, axis=1)
sub = test.groupby('ship').count().reset_index()[['ship']]
sub['pred'] = pred

print(sub['pred'].value_counts(1))
sub['pred'] = sub['pred'].map(type_map_rev)
sub.to_csv('128-cnn-result.csv', index=None, header=None)

0    0.7155
1    0.1690
2    0.1155
Name: pred, dtype: float64


In [194]:
outputs = model(x_test.cuda())
test_feature=pd.DataFrame(outputs[1][0].cpu().detach().numpy())
test_feature.to_csv('test_cnn_feature.csv')

In [195]:
outputs = model(x_train.cuda())
train_feature=pd.DataFrame(outputs[1][0].cpu().detach().numpy())
train_feature.to_csv('train_cnn_feature.csv')

In [456]:
torch.save(model.state_dict(),'model_cnn_state_dict')

In [455]:
torch.save(model,'model_cnn')